In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import matplotlib.pyplot as plt

1. zero-padding 적용

In [2]:
img = torch.FloatTensor([[0.,0.,0.,0.,0.,0.,0.,0.],
                        [0.,2.,0.,2.,1.,3.,2.,0.], 
                        [0.,0.,2.,0.,2.,2.,2.,0.],
                        [0.,1.,0.,1.,3.,1.,1.,0.],
                        [0.,0.,0.,1.,1.,1.,0.,0.],
                        [0.,0.,1.,3.,4.,1.,0.,0.],
                        [0.,0.,1.,0.,0.,5.,2.,0.],
                        [0.,0.,0.,0.,0.,0.,0.,0.]
                        ])
print(img)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 2., 0., 2., 1., 3., 2., 0.],
        [0., 0., 2., 0., 2., 2., 2., 0.],
        [0., 1., 0., 1., 3., 1., 1., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 3., 4., 1., 0., 0.],
        [0., 0., 1., 0., 0., 5., 2., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [3]:
filter = torch.FloatTensor([[1,0,0],
                           [0,1,0],
                           [0,0,1]] 
                          )
print(filter)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


2. filter 적용 후

In [4]:
out = torch.FloatTensor([[4,0,4,3,5,2],
                         [0,5,3,5,4,5],
                         [1,1,4,4,3,3],
                         [1,4,5,3,4,1],
                         [1,1,3,10,4,1],
                         [0,1,1,3,9,3]])
print(out)

tensor([[ 4.,  0.,  4.,  3.,  5.,  2.],
        [ 0.,  5.,  3.,  5.,  4.,  5.],
        [ 1.,  1.,  4.,  4.,  3.,  3.],
        [ 1.,  4.,  5.,  3.,  4.,  1.],
        [ 1.,  1.,  3., 10.,  4.,  1.],
        [ 0.,  1.,  1.,  3.,  9.,  3.]])


3. MaxPooling 적용 후

In [5]:
out = torch.FloatTensor([[5,5,5],
                        [4,5,4],
                        [1,10,9]])
print(out)

tensor([[ 5.,  5.,  5.],
        [ 4.,  5.,  4.],
        [ 1., 10.,  9.]])


## Q2

In [6]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [7]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor())

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor())


data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

2-1)

In [8]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        self.fc1 = torch.nn.Linear(2 * 2 * 128, 625, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))

        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

2-2)  size : (128,2,2) 

In [9]:
input = torch.Tensor(1,1,28,28)
conv1 = torch.nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1)
conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
pool2 =torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

out = conv1(input)
out = pool(out)
out = conv2(out)
out = pool(out)
out = conv3(out)
out = pool2(out)

out.shape

torch.Size([1, 128, 2, 2])

In [10]:
model = CNN()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [11]:
total_batch = len(data_loader)
model.train()    

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.240598843
[Epoch:    2] cost = 0.0616202652
[Epoch:    3] cost = 0.0441167206
[Epoch:    4] cost = 0.0368555374
[Epoch:    5] cost = 0.0277456604
[Epoch:    6] cost = 0.0240929853
[Epoch:    7] cost = 0.0203399286
[Epoch:    8] cost = 0.017216647
[Epoch:    9] cost = 0.0164546594
[Epoch:   10] cost = 0.0141083607
[Epoch:   11] cost = 0.0126078837
[Epoch:   12] cost = 0.010524869
[Epoch:   13] cost = 0.0108740414
[Epoch:   14] cost = 0.00891111512
[Epoch:   15] cost = 0.00998338591


In [13]:
with torch.no_grad():
    model.eval()  

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float()
    Y_test = mnist_test.test_labels

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.984499990940094
